In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import requests
try:
    import statsmodels.api as sm
except:
    !pip install statsmodels
    import statsmodels.api as sm
import seaborn as sns
from scipy.stats import chi2_contingency
import requests
import io

# 新段落

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''
# We will use the method below to import external files

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [ ]:
# importing Sample_Dataset_2014.csv

# Get the file ID from Google Drive share link
file_id = "1VcluNkVfGUdZbW5-66yYy2iqXXM0myww"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Use pandas to read the CSV content
Sample_Dataset_2014 = pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')))

# Display the dataframe
Sample_Dataset_2014.head()

ids        bday enrolldate expgradate Rank                    Major  \
0  43783   3/22/1995                                                       
1  20278    1/1/1995                                          Philosophy   
2  20389  12/31/1994                                                       
3  22820   12/1/1994                             business administration   
4  24559  11/10/1994                                                       

  Gender  Athlete Height  Weight  ... English Reading   Math Writing  \
0      0        0  72.35          ...   88.24    81.5  60.02   81.44   
1      0        0  70.66   179.2  ...   89.45   85.25  70.19   73.27   
2      0        0  70.68  198.52  ...   96.73   86.88   71.2   84.24   
3      1        0         198.34  ...   74.06   88.68  55.89   73.16   
4      1        1  67.43  128.17  ...   82.61    77.3  65.52   80.45   

          State LiveOnCampus HowCommute CommuteTime SleepTime StudyTime  
0      In state            1                                7         1  
1                          1                                5         2  
2      In state                                             8         7  
3      In state            1                                2         6  
4  Out of state            1                                7         3  

[5 rows x 23 columns]

In [ ]:
# Retain only the required columns
selected_columns = ['Gender', 'Athlete', 'Smoking']
df_selected = Sample_Dataset_2014[selected_columns]

# Display the first few rows of the new DataFrame
df_selected.head(10)

Gender  Athlete Smoking
0      0        0       0
1      0        0       0
2      0        0       0
3      1        0        
4      1        1       2
5      0        1       0
6      0        1       0
7      0        1       0
8      1        1       0
9      1        1       0

In [ ]:
# Replace whitespace-only strings with NaN
df_selected.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Drop rows with any 'Null' or empty values
df_cleaned = df_selected.dropna()
#df_cleaned = df_cleaned[df_cleaned.apply(lambda x: x.str.strip() != "").all(axis=1)]

print(df_cleaned.head(10))

shape = df_cleaned.shape
# Printing the dimensions of the DataFrame
print("Number of rows:", shape[0])
print("Number of columns:", shape[1])

   Gender  Athlete Smoking
0       0        0       0
1       0        0       0
2       0        0       0
4       1        1       2
5       0        1       0
6       0        1       0
7       0        1       0
8       1        1       0
9       1        1       0
10      0        0       0
Number of rows: 402
Number of columns: 3


<ipython-input-7-5b51f266ad1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.replace(r'^\s*$', np.nan, regex=True, inplace=True)


In [ ]:
# Sort DataFrame by 'Gender'
df_sorted = df_cleaned.sort_values(by='Gender', ascending=True)  # 0 (male) will be at the top, and 1 (female) will be at the bottom


df_sorted.head(10)

Gender  Athlete Smoking
0        0        0       0
357      0        0       2
355      0        0       0
164      0        1       0
353      0        0       0
167      0        0       0
168      0        0       0
351      0        1       0
171      0        0       1
175      0        1       0

In [ ]:
df_sorted.tail(10)

Gender  Athlete Smoking
288      1        1       0
199      1        0       0
197      1        1       0
196      1        0       0
195      1        1       0
191      1        1       0
189      1        0       1
188      1        0       0
198      1        1       0
434      1        1       1

In [ ]:
# Group by 'Gender', 'Athlete', and 'Smoking' and get the size of each group
table = df_sorted.groupby(['Gender', 'Athlete', 'Smoking']).size().reset_index(name='Count')


# Pivot the table to get a 3D representation
table_pivot = table.pivot_table('Count', ['Gender', 'Athlete'], 'Smoking', fill_value=0)

# Rename the index and columns for better readability
table_pivot.index.set_names(['Gender', 'Athlete'], inplace=True)
table_pivot.columns.name = 'Smoking'
table_pivot.columns = ['Not Smoke', 'Sometimes Smoke', 'Often Smoke']

# Calculate the 'Row Total' for each row
table_pivot['Row Total'] = table_pivot.sum(axis=1)  # Sum across columns for each row

# Calculate the 'Total' row and append it to the DataFrame
total_row = table_pivot.sum(axis=0)

# Display the table
print(table_pivot)



                Not Smoke  Sometimes Smoke  Often Smoke  Row Total
Gender Athlete                                                    
0      0               65               10           23         98
       1               84                3            8         95
1      0               87               17           29        133
       1               61                7            8         76


In [ ]:
# Group by 'Gender', 'Athlete', and 'Smoking' and get the size of each group
grouped = df_sorted.groupby(['Gender', 'Athlete', 'Smoking']).size()

# Initialize a 3D NumPy array to store the counts
table_array = np.zeros((2, 2, 3), dtype=int)  # [2 levels of Gender, 2 levels of Athlete, 3 levels of Smoking]

# Fill the 3D array with the counts from the grouped object
for (gender, athlete, smoking), count in grouped.items():
    # Ensuring the indices are integers before using them as indices in the array
    table_array[int(gender), int(athlete), int(smoking)] = count


# Calculate the Row Total and Column Total
row_total = table_array.sum(axis=2)  # Sum across the 'Smoking' axis
col_total = table_array.sum(axis=(0, 1))  # Sum across the 'Gender' and 'Athlete' axes

# Print the 3D array and the totals
print("3D Table Array:")
print(table_array)
print("\nRow Total:")
print(row_total)
print("\nColumn Total:")
print(col_total)

3D Table Array:
[[[65 10 23]
  [84  3  8]]

 [[87 17 29]
  [61  7  8]]]

Row Total:
[[ 98  95]
 [133  76]]

Column Total:
[297  37  68]


In [ ]:
import numpy as np
from scipy.stats import chi2

# Create a 3-dimensional contingency table (replace this with your own data)
# The table should be a NumPy array with dimensions (rows, columns, layers)
table = table_array

# Calculate the expected frequencies
row_totals = table.sum(axis=(1, 2))
column_totals = table.sum(axis=(0, 2))
layer_totals = table.sum(axis=(0, 1))
total = table.sum()

expected = np.zeros_like(table, dtype=float)

for i in range(table.shape[0]):
    for j in range(table.shape[1]):
        for k in range(table.shape[2]):
            expected[i, j, k] = (row_totals[i] * column_totals[j] * layer_totals[k]) / (total*total)

# Calculate the chi-square statistic
chi2_statistic = np.sum((table - expected)**2 / expected)

# Calculate the degrees of freedom
rows, columns, layers = table.shape
dof = rows*columns*layers-rows-columns-layers+2

# Calculate the p-value
p_value = 1 - chi2.cdf(chi2_statistic, dof)

# Perform the chi-square test
alpha = 0.05  # Significance level
critical_value = chi2.ppf(1 - alpha, dof)

# Display the results
print("Chi-Square Statistic:", chi2_statistic)
print("Degrees of Freedom:", dof)
print("P-value:", p_value)
print("Critical Value:", critical_value)


Chi-Square Statistic: 29.436652083548996
Degrees of Freedom: 7
P-value: 0.0001204210118204152
Critical Value: 14.067140449340169


In [ ]:
# import numpy as np
# from scipy.stats import chi2

# # Create a 3-dimensional contingency table (replace this with your own data)
# # The table should be a NumPy array with dimensions (rows, columns, layers)
# table = np.array([[[50, 30], [20, 10]], [[60, 40], [30, 15]]])

# # Calculate the expected frequencies
# row_totals = table.sum(axis=(1, 2))
# column_totals = table.sum(axis=(0, 2))
# layer_totals = table.sum(axis=(0, 1))
# total = table.sum()

# expected = np.zeros_like(table, dtype=float)

# for i in range(table.shape[0]):
#     for j in range(table.shape[1]):
#         for k in range(table.shape[2]):
#             expected[i, j, k] = (row_totals[i] * column_totals[j] * layer_totals[k]) / (total*total)

# # Calculate the chi-square statistic
# chi2_statistic = np.sum((table - expected)**2 / expected)

# # Calculate the degrees of freedom
# rows, columns, layers = table.shape
# dof = rows*columns*layers-rows-columns-layers+2

# # Calculate the p-value
# p_value = 1 - chi2.cdf(chi2_statistic, dof)

# # Perform the chi-square test
# alpha = 0.05  # Significance level
# critical_value = chi2.ppf(1 - alpha, dof)

# # Display the results
# print("Chi-Square Statistic:", chi2_statistic)
# print("Degrees of Freedom:", dof)
# print("P-value:", p_value)
# print("Critical Value:", critical_value)


In [ ]:
import numpy as np
from scipy.stats import chi2_contingency

# Create a 3-dimensional contingency table (replace this with your own data)
# The table should be a NumPy array with dimensions (rows, columns, layers)
table = table_array

# Perform the chi-square test
res = chi2_contingency(table)

# Display the results
print("Chi-Square Statistic:", res.statistic)
print("Degrees of Freedom:", res.dof)
print("P-value:", res.pvalue)
print("Expected Frequencies Table:")
print(res.expected_freq)

Chi-Square Statistic: 29.436652083548996
Degrees of Freedom: 7
P-value: 0.00012042101182039458
Expected Frequencies Table:
[[[81.93578748 10.20748868 18.75970892]
  [60.65376476  7.55619292 13.88705725]]

 [[88.72839162 11.05370535 20.31491795]
  [65.68205614  8.18261305 15.03831588]]]
